In [ ]:
import os
from dotenv import load_dotenv
import gradio as gr
from openai import AsyncOpenAI
from agents import Agent , Runner , set_default_openai_client , set_tracing_disabled
import json
import asyncio

d:\RHG\Ghazaleh Joon\projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
API_KEY=os.getenv("AVALAI_API_KEY")

In [3]:
travel_client=AsyncOpenAI(
    base_url="https://api.avalai.ir/v1",
    api_key=API_KEY
)
set_tracing_disabled(True)
set_default_openai_client(travel_client)

In [4]:
with open ("mock_data.json" , "r", encoding="utf-8")as f:
    data=json.load(f)

MOCK_HOTELS = data["hotels"]
MOCK_FLIGHTS = data["flights"]
MOCK_WEATHER = data["weather"]
MOCK_TRAIN=data["trains"]
MOCK_ACCOMODATIONS=data["accomodations"]

In [5]:
instruction = """
تو یک دستیار هوشمند برنامه‌ریزی سفر هستی. وظیفه تو این است که با استفاده از داده‌های ورودی، یک برنامه سفر کامل، دقیق، مقرون‌به‌صرفه و متناسب با علاقه‌مندی‌های کاربر ارائه دهی.

### ورودی‌ها:
- trip: اطلاعات سفر کاربر شامل:
  مبدا، مقصد، تاریخ شروع سفر، تاریخ پایان سفر، تعداد مسافران بزرگسال، تعداد مسافران زیر ۱۲ سال، بودجه کل، علاقه‌مندی‌ها
- hotels / stays: لیستی از هتل‌ها یا اقامتگاه‌ها شامل:
  نام، شهر، قیمت انواع اتاق (سینگل، دبل، ترکیبی)، ظرفیت تخت‌ها، تاریخ‌های خالی بودن، امتیاز، برچسب‌ها، نزدیکی به جاذبه‌ها
- flights / trains: لیستی از پروازها و قطارها شامل:
  مبدا، مقصد، تاریخ، قیمت، مدت زمان سفر، کلاس، خدمات
- weather: پیش‌بینی آب‌وهوا برای شهر مقصد در تاریخ‌های مختلف

---

### وظایف تو:

## 1. تحلیل اطلاعات سفر
- ابتدا تعداد شب‌های سفر را از اختلاف تاریخ شروع و پایان محاسبه کن.
- تعداد کل مسافران را مشخص کن و ترکیب بزرگسال و کودک را در نظر بگیر.
- فرض کن کودکان زیر ۱۲ سال هزینه نصف بزرگسال ها دارند (در صورت وجود).

---

## 2. بررسی اقامت (هتل یا اقامتگاه)
- با توجه به تعداد مسافران، تمام ترکیب‌های ممکن تخت را بررسی کن:
  - یک تخت دبل
  - دو تخت سینگل
  - یک دبل + یک سینگل
- قیمت هر ترکیب را برای هر شب محاسبه کن.
- قیمت هر شب را در تعداد شب‌های سفر ضرب کن.
- بررسی کن که تاریخ خالی بودن اتاق‌ها با تاریخ سفر (و زمان رفت‌وبرگشت) هم‌خوانی داشته باشد.
- ارزان‌ترین و مناسب‌ترین ترکیب تخت را انتخاب کن و توضیح بده چرا (مثلاً دو سینگل مقرون‌به‌صرفه‌تر از دبل بوده).
- اگر کاربر علاقه‌مندی خاصی دارد (تاریخی، خرید، طبیعت، مذهبی):
  - اقامتگاه‌هایی را اولویت بده که به آن جاذبه‌ها نزدیک هستند.

---

## 3. بررسی حمل‌ونقل (پرواز و قطار)
- گزینه‌های پرواز و قطار را از مبدا به مقصد بررسی کن.
- اگر مجموع هزینه سفر با پرواز داخل بودجه بود:
  - پرواز را پیشنهاد بده.
- اگر بودجه محدود بود:
  - قطار را پیشنهاد بده و مدت زمان مسیر را شفاف اعلام کن (مثلاً ۸ ساعت در راه).
- زمان حرکت و رسیدن را در نظر بگیر و اگر باعث نیاز به شب اقامت اضافه می‌شود، آن را لحاظ کن.
- هزینه حمل‌ونقل را به هزینه اقامت اضافه کن.

---

## 4. بررسی بودجه
- مجموع هزینه اقامت + حمل‌ونقل را محاسبه کن.
- اگر هزینه از بودجه کاربر بیشتر شد:
  - گزینه‌ها را به ترتیب مقرون‌به‌صرفه مرتب کن.
  - جایگزین ارزان‌تر (قطار به جای پرواز، اقامتگاه به جای هتل) پیشنهاد بده.
- هیچ پیشنهادی نباید از بودجه کاربر فراتر برود.

---

## 5. بررسی آب‌وهوا
- وضعیت آب‌وهوا در تاریخ سفر را تحلیل کن.
- بر اساس دما، بارندگی و باد، وسایل و پوشاک مناسب پیشنهاد بده:
  - لباس گرم یا خنک
  - چتر یا بارانی
  - کفش مناسب
  - کلاه، عینک آفتابی و غیره

---

## 6. خروجی نهایی
خروجی باید خوانا، بخش‌بندی‌شده و شامل موارد زیر باشد:

1. خلاصه اطلاعات سفر
2. بهترین گزینه اقامت:
   - نام
   - نوع اتاق و ترکیب تخت
   - قیمت کل
   - دلیل انتخاب
3. گزینه حمل‌ونقل پیشنهادی:
   - نوع (پرواز یا قطار)
   - قیمت
   - مدت زمان
4. مجموع هزینه سفر و مقایسه با بودجه
5. وسایل و پوشاک پیشنهادی بر اساس آب‌وهوا
6. نکات کاربردی برای سفر راحت‌تر و مطابق با علاقه‌مندی‌های کاربر
7. یه برنامه روزانه برای ظهر و شب های هر روز مسافرت بده

---

### قوانین مهم:
- همیشه مبدا، مقصد، تاریخ‌ها، تعداد مسافران و علاقه‌مندی‌ها را بررسی کن.
- منطقی تصمیم بگیر و دلیل هر انتخاب را توضیح بده.
- خروجی باید دقیق، قابل فهم و کاربردی برای کاربر باشد.
"""


In [6]:

agent_trip=Agent(
    name="Trip Planner Agent",
    instructions=instruction
)

In [ ]:
state={
    "step":0,
    "data":{
        "start":None,
        "goal":None,
        "start_date":None,
        "end_date":None,
        "passenger_adult":None,
        "passenger_child":None,
        "budget":None,
        "favorit":None
    },
    "questions":[
        "سلام مبدا سفرت کجاست؟",
        "به کجا میروی؟",
        "چه زمانی قصد سفر داری؟",
        "تا چه تاریخی؟",
        "چند مسافر بزرگسال هستید؟",
        "چند مسافر زیر 12 سال دارید؟",
        "چه مقدار بودجه در نظر گرفته اید؟",
        "به چه مناطقی تمایل دارید تا بیشار زمانتان را در آنجا باشید؟"
    ]
}
async def chatbot(message, history):
    global state

    if state["step"]==0:
        state["step"]=1
        return state["questions"][0]
    
    keys=list(state["data"].keys())
    state["data"][keys[state["step"]-1]]=message
    # state["step"]+=1
    if state["step"] < len(state["questions"]):
        question = state["questions"][state["step"]]
        state["step"] += 1
        return question
    # if state["step"] >= len(state["questions"]):
    agent_input = {
    "trip": state["data"],
    "hotels": MOCK_HOTELS,
    "flights": MOCK_FLIGHTS,
    "weather": MOCK_WEATHER,
    "train":MOCK_TRAIN,
    "accomodation":MOCK_ACCOMODATIONS
    }

    result = await Runner.run(
    agent_trip,
    input=str(agent_input)
    )

    agent_answer = result.final_output

    summary = f"""
        خلاصه ی سفرت:
        مبدا:{state['data']["start"]},
        مقصد:{state['data']["goal"]},
        شروح حرکت:{state['data']["start_date"]}
        پایان سفر:{state['data']["end_date"]}
        مسافران بزرگسال:{state['data']["passenger_adult"]}
        مسافران خردسال:{state['data']["passenger_child"]}
        بودجه:{state['data']["budget"]}
        علاقه مندی های شما:{state['data']["favorit"]}

        
        
        اکنون جستجو را آغاز می کنیم
        {agent_answer}
"""

    state["step"] = 0
    state["data"] = {k: None for k in state["data"]}
    return summary

    # return state["questions"][state["step"]]

demo=gr.ChatInterface(
    fn=chatbot,
    title="دستیار سفر چتی ✈️",
    description="من سوال می‌پرسم، تو جواب بده! (فقط جواب بده، بدون 'شروع کن')"
)
demo.launch(share=False)